In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import torch
from datasets import load_dataset
from datetime import datetime
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    LlamaForCausalLM
)

from peft import AutoPeftModelForCausalLM, LoraConfig, PeftModel
from trl import SFTTrainer
from accelerate import Accelerator
from accelerate.utils import gather_object
os.environ["NCCL_P2P_DISABLE"] = "1"  #Disabling for dual GPU for RTX4090s
os.environ["NCCL_IB_DISABLE"] = "1"   #Disabling for dual GPU for RTX4090s

In [3]:
# The model that you want to train from the Hugging Face hub
model_name = "../Models/hf_converted_models/Meta-llama-3-8B-Instruct"

# The instruction dataset to use
dataset_name = "../Data/Jsonl/train.jsonl"
test_dataset_name= "../Data/Jsonl/test.jsonl"
# Fine-tuned model name
new_model = "../Models/FineTune/Meta-llama-3-8B-Instruct-finetuned-KCDC"

In [4]:

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16 

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 500

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 6

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None #will default to 1024

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = "auto"

In [5]:
# Load dataset (you can process it here)
dataset = load_dataset('json', data_files={ 'train': [dataset_name], 'test':test_dataset_name}, split="train")

In [6]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    ddp_find_unused_parameters=False,
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

start_train_time = datetime.now()
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)
end_train_time = datetime.now()
print('Train Duration: {}'.format(end_train_time - start_train_time))

Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Step,Training Loss
25,2.077300
50,0.900700
75,0.155800
100,0.028300
125,0.023400
150,0.019900
175,0.021700
200,0.018600
225,0.015700
250,0.016100


Train Duration: 0:21:12.257754


In [7]:
model = AutoPeftModelForCausalLM.from_pretrained(
    new_model,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.


In [8]:
logging.set_verbosity(logging.CRITICAL)

start_time = datetime.now()
# Run text generation pipeline with our next model
prompt = "Write a new presentation on technologies that Min Maung and Lwin Maung normally talk about."
context = """
You are a professional technical public speaker. You write detailed technical abstracts that is engaging and exciting for users."""
final_prompt = f"""<<SYS>>{context}<</SYS>>
[INST]Format should be like this: Title: title of the presentation\n\Description: 3 paragrahs worth of of content[/INST]
[INST]{prompt}[/INST]\n\n"""

print(prompt)
print('Start : {}'.format(datetime.now()))
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2048,top_p=.7,temperature=0.3)
result = pipe(final_prompt)
print(result[0]['generated_text'])
end_time = datetime.now()
print('Inference Duration: {}'.format(end_time - start_time))

Write a new presentation on technologies that Min Maung and Lwin Maung normally talk about.
Start : 2024-06-27 21:16:43.395527
<<SYS>>
You are a professional technical public speaker. You write detailed technical abstracts that is engaging and exciting for users.<</SYS>>
[INST]Format should be like this: Title: title of the presentation
\Description: 3 paragrahs worth of of content[/INST]
[INST]Write a new presentation on technologies that Min Maung and Lwin Maung normally talk about.[/INST]

Title: Revolutionizing IoT Systems: Combining AI and Secure Communication Technologies

Description: In today's interconnected world, IoT systems have become integral to various industries, from manufacturing to healthcare. However, the sheer scale and complexity of these systems have raised critical security concerns. This presentation explores the powerful combination of Artificial Intelligence (AI) and secure communication technologies in transforming IoT systems into secure and reliable infras